In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import streamlit as st


In [2]:
# Load and preprocess dataset
df = pd.read_csv("IMDB Dataset.csv")
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

C:\Users\prabh\AppData\Local\Temp\ipykernel_5148\1754822862.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]


In [13]:
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [4]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

c:\Users\prabh\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
import datetime,time
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping


In [10]:
# TensorBoard callback
log_dir = f"logs/fit/{time.strftime('%Y-%m-%d_%H-%M-%S')}"  # Directory for TensorBoard logs
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# EarlyStopping callback
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [12]:
# Train the model
model.fit(X_train, Y_train, batch_size=64, epochs=5, validation_split=0.2,callbacks=[tensorboard_callback, early_stopping_callback])
model.save('my_movie_review_model.keras')

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 253s 505ms/step - accuracy: 0.8646 - loss: 0.3277 - val_accuracy: 0.8759 - val_loss: 0.2941
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 159s 298ms/step - accuracy: 0.9051 - loss: 0.2374 - val_accuracy: 0.8777 - val_loss: 0.3260
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 133s 266ms/step - accuracy: 0.9181 - loss: 0.2097 - val_accuracy: 0.8785 - val_loss: 0.3018
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 114s 228ms/step - accuracy: 0.9318 - loss: 0.1775 - val_accuracy: 0.8829 - val_loss: 0.3171
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 119s 237ms/step - accuracy: 0.9495 - loss: 0.1363 - val_accuracy: 0.8702 - val_loss: 0.3356
